# Object Detection with YOLOV5

## Install and Import Dependencies

In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 2.5/2.5 MB 7.5 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...


In [1]:
import torch
from matplotlib import pyplot as plt  
import numpy as np
import cv2 as cv
import uuid
import os
import time

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in C:\Users\kdelfino/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-8-13 Python-3.9.15 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
cap = cv.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()

    results = model(frame)

    cv.imshow('YOLO', np.squeeze(results.render()))
    print(results.)

    if cv.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv.destroyAllWindows()